### ELT pipeline
6 steps to extract the data from the excel files into the bigquery datawarehouse where the SQL queries will be perform to answer the business questions.

In [ ]:
# Step 1: Install Required Libraries
!pip install pandas google-cloud-bigquery google-cloud-storage bcchapi

In [ ]:
# Step 2: Upload the service account JSON file
from google.colab import files
uploaded = files.upload()

In step 3 you access my GCP account through a service account to load the excel into a bigquery project:

In [46]:
# Step 3: Authenticate with the service account
import os
import json

service_account_info = json.load(open('sumup_case_sa.json'))
project_id = service_account_info['project_id']

from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_info(service_account_info)

from google.cloud import bigquery
from google.cloud import storage

bq_client = bigquery.Client(credentials=credentials, project=project_id)
storage_client = storage.Client(credentials=credentials, project=project_id)

Steps 4 to 6 extract the data and load it into the datawarehouse.

It is necessary to upload the 3 excel files to Google Colab (drag them to the section with the folder icon in the menu on the left). It could have been made more sophisticated (e.g. loading it from a shared folder in the cloud), but I think it was beyond the focus of the exercise.

In [47]:
# Step 4: Convert Excel Files to CSV
import pandas as pd

excel_files = ['store.xlsx', 'device.xlsx', 'transaction.xlsx']
csv_files = ['stores.csv', 'devices.csv', 'transactions.csv']

for excel_file, csv_file in zip(excel_files, csv_files):
    df = pd.read_excel(excel_file)
    df.to_csv(csv_file, index=False)

In [48]:
# Step 5: Upload CSV Files to Google Cloud Storage
bucket_name = "sumup_case"
bucket = storage_client.bucket(bucket_name)

for file in csv_files:
    blob = bucket.blob(file)
    blob.upload_from_filename(file)


In [ ]:
# Step 6: Load CSV Data into BigQuery
def load_data_from_gcs_to_bq(dataset_name, table_name, gcs_uri):
    dataset_ref = bq_client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)

    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # If the tables already exist, it rewrites them instead of duplicating the data, so you can run the script several times without problem
    )

    load_job = bq_client.load_table_from_uri(
        gcs_uri,
        table_ref,
        job_config=job_config
    )
    load_job.result()  # Waits for the job to complete

    print(f"Loaded {load_job.output_rows} rows into {dataset_name}:{table_name}")

dataset_name = 'SumUp_Case'
bucket_uri = f"gs://{bucket_name}/"

load_data_from_gcs_to_bq(dataset_name, 'stores', f"{bucket_uri}stores.csv")
load_data_from_gcs_to_bq(dataset_name, 'devices', f"{bucket_uri}devices.csv")
load_data_from_gcs_to_bq(dataset_name, 'transactions', f"{bucket_uri}transactions.csv")


### UF Value ingestion:
The time window select for this series is YTD.

In [ ]:
# Step 7: Fetch Data from Banco Central de Chile API
import bcchapi

# datetime imports for building the YTD logic for the UF series:
from datetime import datetime

# Api credentials:
siete = bcchapi.Siete(file="credenciales.txt")

# Get the current date
today = datetime.today()

# Set the first day of the year
first_day_of_year = datetime(today.year, 1, 1).strftime('%Y-%m-%d')

# Set today
today_str = today.strftime('%Y-%m-%d')

# Create the table using the UF series code. The output is a panda dataframe object:
uf_df = siete.cuadro(
    series=["F073.UFF.PRE.Z.D"],
    nombres=["uf"],
    desde=first_day_of_year,
    hasta=today_str
)

# Save the DataFrame to CSV
uf_csv_file = 'uf_data.csv'
uf_df.to_csv(uf_csv_file, index=True)

# Step 8: Upload the UF CSV to Google Cloud Storage
uf_blob = bucket.blob(uf_csv_file)
uf_blob.upload_from_filename(uf_csv_file)

# Step 9: Load the UF CSV Data into BigQuery
load_data_from_gcs_to_bq(dataset_name, 'uf_data', f"{bucket_uri}{uf_csv_file}")
